In [135]:
import pandas as pd
import numpy as np

In [136]:
df_train = pd.read_csv('../analysis/train.csv')
df_label_train = pd.read_csv('../analysis/train_label.csv')

In [137]:
block_id_anomalous = df_label_train[df_label_train['anomalous'] == 1]['block_id'].values

In [138]:
df_anomalous = df_train[df_train['block_id'].isin(block_id_anomalous)]

In [139]:
df_desc = df_anomalous.describe()

In [140]:
df_desc.head()

,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
count,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000,1310.000000
mean,8895.374046,2.447115,48.171005,52.139468,44.318828,630.680153,74.994393,13.799454,16.153960,15.449343,15.131636
std,5470.346861,0.211745,1.704151,1.505023,1.509306,38.456865,9.954861,0.999794,0.358564,0.381017,0.223266
min,197.000000,0.000000,40.104160,45.182290,40.190970,200.558441,0.000000,3.009259,14.712438,14.456552,14.147890
25%,4437.000000,2.450637,47.178820,51.176517,43.402775,629.629600,71.200263,13.418690,15.957750,15.183740,15.053530


In [162]:
mean = df_desc.loc['mean'].values[1:]
std = df_desc.loc['std'].values[1:]

In [163]:
data = []
for m, s in zip(mean, std):
    sample = np.random.normal(m, s, 40000).reshape(-1, 1)
    data.append(sample)
data = np.hstack(data)
data.shape

(40000, 10)

In [164]:
cols = list(df_desc.columns)[1:]
df_new_data = pd.DataFrame(columns=cols, data=data)

In [165]:
tmp = df_anomalous[['timestamp', 'block_id']][:1000].reset_index(drop=True)

In [166]:
tmp = pd.concat([tmp] * 40, axis=0).reset_index(drop=True)

In [167]:
df_new_data = pd.concat([tmp, df_new_data], axis=1)

In [168]:
df_new_data.head()

,timestamp,block_id,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09
0,2018-04-02 08:50:00,197,2.649770,50.761822,51.660732,46.581518,600.222868,63.688093,13.765963,16.570691,16.023164,15.160535
1,2018-04-02 08:51:00,197,2.474069,50.435159,51.927784,44.268270,651.176558,68.183512,13.371503,15.795449,15.701983,15.069100
2,2018-04-02 08:52:00,197,2.750751,46.423771,50.520977,42.427946,558.525639,63.419210,14.849667,16.694509,15.338461,14.941691
3,2018-04-02 08:53:00,197,2.480022,45.541142,51.240268,43.942311,655.668650,80.963593,15.296042,16.088853,15.078666,14.823551
4,2018-04-02 08:54:00,197,2.523577,49.543614,52.862617,45.709519,647.056826,68.814586,14.014246,15.747767,15.112354,15.193404


In [169]:
df_new_data.to_csv('../analysis/new_data/new_data.csv', index=False)

In [170]:
block_id_new = np.unique(df_new_data['block_id'])
block_id_new, block_id_new.shape

(array([  197,   214,   270,   579,   606,   618,   647,   677,  1293,
         1315,  1316,  1319,  1320,  1321,  1322,  2035,  2668,  2707,
         2771,  2785,  2812,  2863,  2865,  2902,  2907,  4086,  4087,
         4106,  4121,  4140,  4375,  4419,  4437,  4438,  4439,  4440,
         4461,  4499,  4515,  4528,  4534,  5737,  5784,  5795,  5796,
         5797,  5850,  5926,  5934,  5935,  5971,  6046,  6994,  7007,
         7046,  7119,  7128,  7134,  7141,  7222,  7226,  7292,  7295,
         8811,  8870,  8871,  8906,  8924,  8952,  9010,  9013,  9029,
         9202,  9215,  9304,  9305,  9306,  9307,  9316, 10493, 10541,
        10555, 10607, 10865, 11060, 11445, 11978, 12751, 12798, 12830,
        12835, 12845, 12897, 12931, 13353, 13391, 13398, 13666, 13696,
        13705]),
 (100,))

In [171]:
df_label_anomalous = df_label_train[df_label_train['block_id'].isin(block_id_anomalous)]

In [172]:
df_new_data_label = df_label_anomalous[df_label_anomalous['block_id'].isin(block_id_new)]

In [176]:
df_new_data_label = pd.concat([df_new_data_label]*40, axis=0)

In [177]:
df_new_data_label.to_csv('../analysis/new_data/df_new_data_label.csv', index=False)

In [178]:
df_new_data_label

,block_id,anomalous
40,197,1
54,214,1
95,270,1
113,579,1
134,606,1
...,...,...
3159,13391,1
3164,13398,1
3271,13666,1
3293,13696,1


In [159]:
train = pd.read_csv('../analysis/train.csv')
val = pd.read_csv('../analysis/val.csv')
target_train = pd.read_csv('../analysis/train_label.csv')
target_val = pd.read_csv('../analysis/val_label.csv')['anomalous'].to_numpy()

# Add feature
train_suffle = pd.read_csv('../analysis/suffle/train_suffle_100.csv')
target_train_shuffle = pd.read_csv('../analysis/suffle/train_label_suffle_100.csv')

train_new_data = pd.read_csv('../analysis/new_data/new_data.csv')
target_new_data = pd.read_csv('../analysis/new_data/df_new_data_label.csv')

# Combine
train = pd.concat([train, train_new_data, train_suffle], axis=0)
target_train = pd.concat([target_train,target_new_data, target_train_shuffle], axis=0)

target_train = target_train['anomalous'].to_numpy()

In [160]:
len(train), len(val)

(69340, 11570)

In [161]:
len(train_new_data), len(train_suffle)

(10000, 13100)